In [8]:
import math
import pandas as pd
import numpy as np
import torch
import pickle

import arviz as az


from sbmfi.inference.flow_trainer import flow_constructor, flow_trainer


import arviz as az
import matplotlib.pyplot as plt

from sbmfi.models.small_models import spiro
from sbmfi.core.polytopia import (
    extract_labelling_polytope, 
    transform_polytope_keep_transform, 
    round_polytope_keep_ellipsoid,
    LabellingPolytope,
    sample_polytope,
    MarkovTransition,
    V_representation
)
from sbmfi.core.coordinater import FluxCoordinateMapper
from sbmfi.priors.mog import MixtureOfGaussians
from sbmfi.inference.arxiv_polytope_plotter import plot_dataframes

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [9]:
fcm  = pickle.load(open('spiro_flux_coordinate_mapper.p', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'spiro_flux_coordinate_mapper.p'

In [1]:
DEVICE = 'cuda:0'
RESCALE_VAL = 1.0

In [2]:
mog_rounded_theta = torch.as_tensor(
    az.extract(mog_data, combined=True,var_names='theta', keep_dataset=False, rng=2).values
).T
fluxes = fcm.map_theta_2_fluxes(mog_rounded_theta, return_thermo=False, rescale_val=None, pandalize=True)
mog_cylinder_theta = fcm.map_fluxes_2_theta(fluxes, coordinate_id='cylinder', rescale_val=RESCALE_VAL, is_thermo=False)
mog_cylinder_theta_df = fcm.map_fluxes_2_theta(fluxes, coordinate_id='cylinder', rescale_val=RESCALE_VAL, is_thermo=False, pandalize=True)


if torch.cuda.is_available() and (DEVICE != 'cpu'):
    mog_cylinder_theta=mog_cylinder_theta.to(DEVICE)

batch_size = 1024 * 12
dataset = torch.utils.data.TensorDataset(mog_cylinder_theta)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

NameError: name 'torch' is not defined

In [3]:
mog_cylinder_theta_df.head(2)

NameError: name 'mog_cylinder_theta_df' is not defined

In [4]:
theta1 = 'phi'
theta2 = 'C_rref_0'
theta2 = 'R'
# theta2 = 'C_rref_1'
buffer = 0.1

cylinder_range = (-(RESCALE_VAL + buffer), (RESCALE_VAL + buffer))
xax = hv.Dimension(theta1, range=cylinder_range)
yax = hv.Dimension(theta2, range=cylinder_range)

hv.Bivariate(
    mog_cylinder_theta_df.sample(10000), kdims=[xax, yax]
).opts(
    filled=True, alpha=1.0, cmap='Blues',fontsize=mog_plotter._FONTSIZES, bandwidth=0.1, show_grid=True, **mog_plotter._size_opts()
)

NameError: name 'hv' is not defined

In [5]:
# from sbmfi.core.linalg import LinAlg

# class Rev_KLD():
#     def __init__(
#         self,
#         fcm: FluxCoordinateMapper,
#         density: torch.distributions.Distribution,
#         rescale_val=1.0,
#     ):
#         self._rescale = rescale_val
#         samples = density.sample(torch.Size((2,)))
#         self._la = device_linalg = LinAlg(backend='torch', device=str(samples.device))
#         self._fcm = fcm.to_linalg(device_linalg)
#         self._density = density
#         self._n_train = 0

#     def log_prob(self, z, context=None):
#         # with torch.no_grad():
#         self._n_train += z.shape[:-1].numel()
#         ball = self._fcm.map_cylinder_2_ball(z, rescale_val=self._rescale)
#         rounded = self._fcm.map_ball_2_rounded(ball)
#         return self._density.log_prob(rounded)

# device_target = target.copy_to(DEVICE)
# rkld = Rev_KLD(fcm, device_target)

# cyl = torch.rand(torch.Size((4, K))) * 2 - 1
# rkld.log_prob(cyl.to(DEVICE))

In [6]:
# prior_flow = flow_constructor(
#     fcm=fcm,
#     coordinate_id='cylinder',
#     rescale_val=RESCALE_VAL,
#     log_xch=False,
#     embedding_net=None,
#     num_context_channels=None,
#     autoregressive=True,
#     num_blocks=4,
#     num_hidden_channels=64,
#     num_bins=30,
#     dropout_probability=0.01,
#     num_transforms=10,
#     init_identity=True,
#     mixing_id='shuffle',
#     use_lu=True,
#     p=rkld,
#     device=DEVICE,
# )

# # loss = prior_flow.reverse_kld(num_samples=5)


In [ ]:
# optimizer = torch.optim.Adam(prior_flow.parameters(), lr=4e-3, weight_decay=1e-3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1)
# steps = 0
# losses=[]

In [ ]:
# torch.save(prior_flow.state_dict(), "prior_flow.pt")
prior_flow.load_state_dict(torch.load("prior_flow.pt"))

In [ ]:
fig, ax = plot_dataframes(
    vertices=v_representation,
    samples=flow_rounded,
    points=means_df,
    x_col='R_h_out', 
    y_col='R_bm',
    x_label='X-axis', 
    y_label='Y-axis',
    label1='Polytope vertices', 
    label2='Samples Convex Hull',
    label3='Samples KDE Density', 
    label4='Additional Points',
    kde_threshold=1e-8,  # informational only (not directly used in sns.kdeplot)
    bw_method=None,
    bw_adjust=0.6,
    levels=8,
    font_dict=None,
    vertices_fill_color='#ffec58',
    samples_fill_color='#e02450',
    figsize=(6, 4),
    show_legend=True,
    legend_loc='best'
)
plt.show()

In [ ]:
diff = log_p - tot_log_q
ws = torch.exp(diff)
KL_Z = ws.mean()
ESS = ws.sum()**2 / (ws**2).sum()
ESS, KL_Z

In [ ]:
tot_log_q = log_q - torch.log(abs(dets)) 
log_p = target.log_prob(flow_rounded)

In [ ]:
(tot_log_q + log_p).mean() # torch.log(KL_Z)